In [1]:
import pandas as pd
data = pd.read_csv("bbc-text.csv")
data.head

<bound method NDFrame.head of            category                                               text
0              tech  tv future in the hands of viewers with home th...
1          business  worldcom boss  left books alone  former worldc...
2             sport  tigers wary of farrell  gamble  leicester say ...
3             sport  yeading face newcastle in fa cup premiership s...
4     entertainment  ocean s twelve raids box office ocean s twelve...
...             ...                                                ...
2220       business  cars pull down us retail figures us retail sal...
2221       politics  kilroy unveils immigration policy ex-chatshow ...
2222  entertainment  rem announce new glasgow concert us band rem h...
2223       politics  how political squabbles snowball it s become c...
2224          sport  souness delight at euro progress boss graeme s...

[2225 rows x 2 columns]>

In [2]:
from sklearn.preprocessing import LabelEncoder
df2 = pd.DataFrame()
df2['text'] = data['text']
df2['label'] = LabelEncoder().fit_transform(data['category'])
df2['label']

0       4
1       0
2       3
3       3
4       1
       ..
2220    0
2221    2
2222    1
2223    2
2224    3
Name: label, Length: 2225, dtype: int32

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
df2['text'] = df2['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df2['text'].head()
df2.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kareema.batool\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,label
0,tv future hands viewers home theatre systems p...,4
1,worldcom boss left books alone former worldcom...,0
2,tigers wary farrell gamble leicester say rushe...,3
3,yeading face newcastle fa cup premiership side...,3
4,ocean twelve raids box office ocean twelve cri...,1


In [4]:
freq = pd.Series(' '.join(df2['text']).split()).value_counts()[-10:]
df2['text'] = df2['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df2['text'].head()

0    tv future hands viewers home theatre systems p...
1    worldcom boss left books alone former worldcom...
2    tigers wary farrell gamble leicester say rushe...
3    yeading face newcastle fa cup premiership side...
4    ocean twelve raids box office ocean twelve cri...
Name: text, dtype: object

In [5]:
import numpy as np
import spacy
from tqdm import tqdm
import re
import time 
import pickle
pd.set_option('display.max_colwidth',200)


ModuleNotFoundError: No module named 'spacy'

In [6]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15


ERROR: Could not find a version that satisfies the requirement tensorflow==1.15
ERROR: No matching distribution found for tensorflow==1.15


ERROR: Could not find a version that satisfies the requirement tensorflow_text==1.15
ERROR: No matching distribution found for tensorflow_text==1.15


In [7]:
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)



Failed to import tensorflow. Please note that tensorflow is not installed by default when you install tensorflow_hub. This is so that users can decide which tensorflow package to use. To use tensorflow_hub, please install a current version of tensorflow by following the instructions at https://tensorflow.org/install and https://tensorflow.org/hub/installation.




ModuleNotFoundError: No module named 'tensorflow'

In [4]:
from sklearn import preprocessing
from tensorflow import keras
y = list(df2['label'])
x = list(df2['text'])
le = preprocessing.LabelEncoder()
le.fit(y)
def encode(le, labels):
  enc = le.transform(labels)
  return keras.utils.to_categorical(enc)

def decode(le, one_hot):
  dec = np.argmax(one_hot, axis = 1)
  return le.inverse_transform(dec)

NameError: ignored

In [12]:
x_enc = x
y_enc = encode (le, y) 
print(x_enc)
y_enc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [13]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size = 0.2, random_state = 42)
y_train.shape


(1780, 5)

In [14]:
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [ ]:
def ELMoEmbedding(x):
  return embed(tf.squeeze(tf.cast(x, tf.string)), signature = 'default', as_dict = True)["default"]


input_text = Input(shape = (1,), dtype = tf.string)
embedding = Lambda(ELMoEmbedding, output_shape = (1024, ))(input_text)
dense = Dense(256, activation = 'relu')(embedding)
pred = Dense(5, activation = 'softmax')(dense)
model = Model(inputs = [input_text], outputs = pred)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(x_train, y_train, epochs = 1, batch_size = 1000)
  model.save_weights('./elmo-model.h5')




INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 1780 samples


In [2]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.save_weights('./elmo-model.h5')
  predicts = model.predict(x_test, batch_size = 16)

y_test = decode(le, y_test)
y_preds = decode (le, predicts)

NameError: ignored

In [2]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test, y_preds))
print(metrics.classification_report(y_test, y_preds))
from sklearn.metrics import accuracy_score
print("The accuracy of ELMo achieved is : ", accuracy_score(y_test, y_preds))

NameError: ignored